In [1]:
import os
import librosa
import numpy as np
import math
from tqdm import tqdm

# ===============================
# Config (DO NOT CHANGE casually)
# ===============================
SAMPLE_RATE = 16000
FRAME_SIZE = 160              # 10 ms
SEGMENT_LEN = 64000           # ✅ MUST be multiple of FRAME_SIZE

IN_DIR = "/kaggle/input/la-test/LA/ASVspoof2019_LA_dev/flac"
OUT_DIR = "/kaggle/working/CS_dev"

os.makedirs(OUT_DIR, exist_ok=True)

# ===============================
# CS Utilities
# ===============================
def circular_shift(arr, shift):
    return np.roll(arr, shift)

def least_number(a, b):
    g = math.gcd(a, b)
    return a // g, b // g

def generate_matrix(N, M, P, Q):
    row = np.concatenate([np.ones(M), np.zeros(N - M)])
    rows = []
    for _ in range(M):
        rows.append(row)
        row = circular_shift(row, M)
    return np.array(rows)

def apply_cs(audio, sensing_matrix, frame_size, segment_len):
    """
    🔥 FIXED-LENGTH CS
    """
    num_frames = segment_len // frame_size
    audio = audio[:num_frames * frame_size]

    return np.concatenate([
        sensing_matrix @ audio[i * frame_size:(i + 1) * frame_size]
        for i in range(num_frames)
    ])

# ===============================
# Build sensing matrix ONCE
# ===============================
M = N = FRAME_SIZE
P, Q = least_number(M, N)
S = generate_matrix(N, M, P, Q)

# ===============================
# Process DEV files
# ===============================
processed = 0

for f in tqdm(os.listdir(IN_DIR), desc="CS preprocessing (DEV)"):
    if not f.endswith(".flac"):
        continue

    utt = f.replace(".flac", "")
    audio_path = os.path.join(IN_DIR, f)

    audio, _ = librosa.load(audio_path, sr=SAMPLE_RATE)

    # 🔥 FORCE FIXED SEGMENT LENGTH
    if len(audio) < SEGMENT_LEN:
        audio = np.pad(audio, (0, SEGMENT_LEN - len(audio)))
    else:
        audio = audio[:SEGMENT_LEN]

    # 🔥 APPLY FIXED CS
    cs_audio = apply_cs(audio, S, FRAME_SIZE, SEGMENT_LEN)

    # 🔥 SAVE FLOAT32
    np.save(
        os.path.join(OUT_DIR, utt + ".npy"),
        cs_audio.astype(np.float32)
    )
    processed += 1

print(f"DEV CS preprocessing complete.")
print(f"Files processed: {processed}")
print(f"Saved to: {OUT_DIR}")


CS preprocessing (DEV): 100%|██████████| 24986/24986 [05:44<00:00, 72.51it/s]

DEV CS preprocessing complete.
Files processed: 24986
Saved to: /kaggle/working/CS_dev


In [2]:
import os
import librosa
import numpy as np
import math
from tqdm import tqdm

# ===============================
# Config (MUST MATCH DEV + TRAINING)
# ===============================
SAMPLE_RATE = 16000
FRAME_SIZE = 160              # 10 ms
SEGMENT_LEN = 64000           # MUST be multiple of FRAME_SIZE

IN_DIR = "/kaggle/input/la-test/LA/ASVspoof2019_LA_train/flac"
OUT_DIR = "/kaggle/working/CS_train"

os.makedirs(OUT_DIR, exist_ok=True)

# ===============================
# CS Utilities
# ===============================
def circular_shift(arr, shift):
    return np.roll(arr, shift)

def least_number(a, b):
    g = math.gcd(a, b)
    return a // g, b // g

def generate_matrix(N, M, P, Q):
    row = np.concatenate([np.ones(M), np.zeros(N - M)])
    rows = []
    for _ in range(M):
        rows.append(row)
        row = circular_shift(row, M)
    return np.array(rows)

def apply_cs(audio, sensing_matrix, frame_size, segment_len):
    """
    🔥 FIXED-LENGTH CS (CRITICAL)
    """
    num_frames = segment_len // frame_size
    audio = audio[:num_frames * frame_size]

    return np.concatenate([
        sensing_matrix @ audio[i * frame_size:(i + 1) * frame_size]
        for i in range(num_frames)
    ])

# ===============================
# Build sensing matrix ONCE
# ===============================
M = N = FRAME_SIZE
P, Q = least_number(M, N)
S = generate_matrix(N, M, P, Q)

# ===============================
# Process TRAIN files
# ===============================
processed = 0

for f in tqdm(os.listdir(IN_DIR), desc="CS preprocessing (TRAIN)"):
    if not f.endswith(".flac"):
        continue

    utt = f.replace(".flac", "")
    audio_path = os.path.join(IN_DIR, f)

    audio, _ = librosa.load(audio_path, sr=SAMPLE_RATE)

    # 🔥 FORCE FIXED SEGMENT LENGTH
    if len(audio) < SEGMENT_LEN:
        audio = np.pad(audio, (0, SEGMENT_LEN - len(audio)))
    else:
        audio = audio[:SEGMENT_LEN]

    # 🔥 APPLY FIXED CS
    cs_audio = apply_cs(audio, S, FRAME_SIZE, SEGMENT_LEN)

    # 🔥 SAVE FLOAT32
    np.save(
        os.path.join(OUT_DIR, utt + ".npy"),
        cs_audio.astype(np.float32)
    )
    processed += 1

print(f"TRAIN CS preprocessing complete.")
print(f"Files processed: {processed}")
print(f"Saved to: {OUT_DIR}")

CS preprocessing (TRAIN): 100%|██████████| 25380/25380 [05:33<00:00, 76.01it/s]

TRAIN CS preprocessing complete.
Files processed: 25380
Saved to: /kaggle/working/CS_train


In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_curve
from tqdm import tqdm
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

SEGMENT_LEN = 16000          # MUST match CS preprocessing
BATCH_SIZE = 16              # now we can increase
EPOCHS = 30
LR = 1e-4

CS_TRAIN_DIR = "/kaggle/working/CS_train"
CS_DEV_DIR   = "/kaggle/working/CS_dev"

os.makedirs("checkpoints", exist_ok=True)
class ASVspoofCSDataset(Dataset):
    def __init__(self, cs_dir, protocol_path):
        self.cs_dir = cs_dir
        self.items = []

        available = set(
            os.path.splitext(f)[0]
            for f in os.listdir(cs_dir)
            if f.endswith(".npy")
        )

        skipped = 0
        with open(protocol_path) as f:
            for line in f:
                parts = line.strip().split()
                utt = parts[1]
                label = 1 if parts[-1] == "spoof" else 0

                if utt not in available:
                    skipped += 1
                    continue

                self.items.append((utt, label))

        print(f"[Dataset] Loaded {len(self.items)} | Skipped {skipped}")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        utt, label = self.items[idx]
        audio = np.load(os.path.join(self.cs_dir, utt + ".npy"))
    
        # 🔥 FORCE FIXED LENGTH
        if len(audio) < SEGMENT_LEN:
            audio = np.pad(audio, (0, SEGMENT_LEN - len(audio)))
        elif len(audio) > SEGMENT_LEN:
            audio = audio[:SEGMENT_LEN]
    
        return torch.tensor(audio, dtype=torch.float32), torch.tensor(label)
    

        return torch.tensor(audio, dtype=torch.float32), torch.tensor(label)
class RawFormerL(nn.Module):
    def __init__(self):
        super().__init__()

        self.frontend = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=251, stride=5, padding=125),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.3),
            nn.MaxPool1d(3),          # 🔥 ADD THIS
        
            nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.3),
            nn.MaxPool1d(3)           # 🔥 ADD THIS
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=256,
            nhead=8,
            dim_feedforward=1024,
            dropout=0.1,
            batch_first=True
        )

        self.transformer = nn.TransformerEncoder(
            encoder_layer, num_layers=4
        )

        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = x.unsqueeze(1)         # (B,1,T)
        x = self.frontend(x)       # (B,C,T')
        x = x.permute(0, 2, 1)     # (B,T',C)

        x = self.transformer(x)

        mean = x.mean(dim=1)
        std = x.std(dim=1)
        stats = torch.cat([mean, std], dim=1)

        return self.fc(stats)
def compute_eer(labels, scores):
    fpr, tpr, _ = roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr
    eer = fpr[np.nanargmin(np.abs(fnr - fpr))]
    return eer * 100
def compute_min_tDCF(scores, labels,
                     Ptar=0.01,
                     Cmiss=1,
                     Cfa=1):
    """
    scores: spoof probabilities (higher = spoof)
    labels: 1 = spoof, 0 = bonafide
    """

    fpr, tpr, _ = roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr

    # CM miss = spoof classified as bonafide
    # CM false alarm = bonafide classified as spoof
    tDCF = Cmiss * Ptar * fnr + Cfa * (1 - Ptar) * fpr

    return np.min(tDCF)
train_set = ASVspoofCSDataset(
    CS_TRAIN_DIR,
    "/kaggle/input/la-test/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt"
)

dev_set = ASVspoofCSDataset(
    CS_DEV_DIR,
    "/kaggle/input/la-test/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt"
)

train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

dev_loader = DataLoader(
    dev_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
model = RawFormerL().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()
best_eer = 100.0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for x, y in pbar:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            out = model(x)
            loss = criterion(out, y)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    # -------- Evaluation --------
    model.eval()
    scores, labels = [], []

    with torch.no_grad():
        for x, y in tqdm(dev_loader, desc="Evaluating", leave=False):
            x = x.to(DEVICE)
            probs = torch.softmax(model(x), dim=1)[:, 1]

            scores.extend(probs.cpu().numpy())
            labels.extend(y.numpy())

    scores = np.array(scores)
    labels = np.array(labels)

    eer = compute_eer(labels, scores)
    min_tdcf = compute_min_tDCF(scores, labels)

    print(
        f"\nEpoch {epoch+1} | "
        f"Avg Loss {total_loss/len(train_loader):.4f} | "
        f"EER {eer:.2f}% | "
        f"min-tDCF {min_tdcf:.4f}\n"
    )

    # -------- Save Best --------
    if eer < best_eer:
        best_eer = eer
        torch.save(
            {
                "epoch": epoch + 1,
                "model_state": model.state_dict(),
                "eer": eer,
                "min_tdcf": min_tdcf
            },
            "/kaggle/working/checkpoints/rawformerL_cs_best.pth"
        )

print("Training complete. Best EER:", best_eer)



Using device: cuda
[Dataset] Loaded 25380 | Skipped 0
[Dataset] Loaded 24844 | Skipped 0


/tmp/ipykernel_25/3411099146.py:162: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/30:   0%|          | 0/1587 [00:00<?, ?it/s]/tmp/ipykernel_25/3411099146.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/30: 100%|██████████| 1587/1587 [00:37<00:00, 42.67it/s, loss=1.0482]



Epoch 1 | Avg Loss 0.2827 | EER 23.08% | min-tDCF 0.0065



Epoch 2/30:   0%|          | 0/1587 [00:00<?, ?it/s]/tmp/ipykernel_25/3411099146.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 2/30: 100%|██████████| 1587/1587 [00:36<00:00, 42.95it/s, loss=0.0209]



Epoch 2 | Avg Loss 0.2337 | EER 13.93% | min-tDCF 0.0066



...


Epoch 28 | Avg Loss 0.1677 | EER 11.97% | min-tDCF 0.0073



Epoch 29/30:   0%|          | 0/1587 [00:00<?, ?it/s]/tmp/ipykernel_25/3411099146.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 29/30: 100%|██████████| 1587/1587 [00:38<00:00, 40.99it/s, loss=0.2179]



Epoch 29 | Avg Loss 0.1677 | EER 12.24% | min-tDCF 0.0062



Epoch 30/30:   0%|          | 0/1587 [00:00<?, ?it/s]/tmp/ipykernel_25/3411099146.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 30/30: 100%|██████████| 1587/1587 [00:38<00:00, 41.08it/s, loss=0.0431]
                                                               


Epoch 30 | Avg Loss 0.1693 | EER 11.85% | min-tDCF 0.0052

Training complete. Best EER: 11.067503924646783


In [4]:
import zipfile
import os

zip_path = "/kaggle/working/checkpoints.zip"
folder_path = "/kaggle/working/checkpoints"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            arcname = os.path.relpath(full_path, folder_path)
            zipf.write(full_path, arcname)

print("Zipped to:", zip_path)

Zipped to: /kaggle/working/checkpoints.zip
